In [1]:
import os
import requests
import pandas as pd
import ast
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from fpdf import FPDF
from PIL import Image
from PyPDF2 import PdfMerger, PdfReader, PdfWriter
import shutil

In [2]:
# Función para extraer el nombre del anime desde la URL
def extract_anime_name(url):
    parts = url.split('/manga/')
    if len(parts) > 1:
        name = parts[1].split('/')[0]
        return name.replace('-', ' ').title()
    return None

def extract_segmento_url(url, x, i):
    parts = url.split('/')
    if len(parts) > 1:
        return '/'.join(parts[:6]) + '/' + x + '/' + i
    return None

def descargar_imagen(ruta_guardado, url_imagen, numero_imagen):
    response = requests.get(url_imagen)
    if response.status_code == 200 and 'image' in response.headers.get('content-type', ''):
        os.makedirs(ruta_guardado, exist_ok=True)
        image_name = f'{numero_imagen}.jpg'
        image_path = os.path.join(ruta_guardado, image_name)
        with open(image_path, 'wb') as f:
            f.write(response.content)
        return image_path
    else:
        print(f'Error al descargar la imagen. Código de estado: {response.status_code}, Tipo de contenido: {response.headers.get("content-type")}')
        return None

def es_imagen_valida(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError) as e:
        print(f'Archivo de imagen inválido o corrupto: {image_path}. Error: {e}')
        return False

def convertir_a_jpeg(image_path):
    try:
        with Image.open(image_path) as img:
            if img.format != 'JPEG':
                rgb_im = img.convert('RGB')
                rgb_im.save(image_path, 'JPEG', quality=100)
                print(f'Imagen convertida a JPEG: {image_path}')
    except Exception as e:
        print(f'Error al convertir la imagen: {image_path}. Error: {e}')

def crear_pdf(ruta_guardado, lista_imagenes, anime_name, capitulo):
    pdf = FPDF()
    for image_path in lista_imagenes:
        if es_imagen_valida(image_path):
            convertir_a_jpeg(image_path)
            pdf.add_page()
            pdf.image(image_path, 0, 0, 210, 297)  # Ajusta el tamaño según sea necesario
        else:
            print(f'Imagen omitida debido a errores: {image_path}')
            os.remove(image_path)
    pdf_name = f"{anime_name}_capitulo_{capitulo}.pdf".replace(" ", "_")
    pdf_path = os.path.join(ruta_guardado, pdf_name)
    pdf.output(pdf_path)
    print(f'PDF guardado en: {pdf_path}')

def mover_pdfs_a_carpeta(ruta_base):
    ruta_pdfs = os.path.join(ruta_base)
    ruta_pdfs_combinados = os.path.join(ruta_base, 'pdfs_combinados')
    os.makedirs(ruta_pdfs_combinados, exist_ok=True)

    pdfs = [f for f in os.listdir(ruta_pdfs) if f.endswith('.pdf')]
    for pdf in pdfs:
        pdf_path = os.path.join(ruta_pdfs, pdf)
        shutil.move(pdf_path, os.path.join(ruta_pdfs_combinados, pdf))

    return ruta_pdfs_combinados

def combinar_pdfs(ruta_pdfs_combinados, anime_name):
    pdfs = [f for f in os.listdir(ruta_pdfs_combinados) if f.endswith('.pdf')]
    pdfs.sort()  # Ordenar por nombre, asumiendo que los nombres incluyen números de capítulo

    writer = PdfWriter()

    for pdf in pdfs:
        pdf_path = os.path.join(ruta_pdfs_combinados, pdf)
        reader = PdfReader(pdf_path)

        # Agregar una página de separación
        page_separacion = PdfWriter()
        page_separacion.add_blank_page(width=210, height=297)
        writer.add_page(page_separacion.pages[0])

        for page in reader.pages:
            writer.add_page(page)

    pdf_combined_name = f"{anime_name}_todos_los_capitulos.pdf".replace(" ", "_")
    pdf_combined_path = os.path.join(ruta_pdfs_combinados, pdf_combined_name)
    with open(pdf_combined_path, 'wb') as f:
        writer.write(f)

    print(f'PDF combinado guardado en: {pdf_combined_path}')

def procesar_dataset(ruta_dataset):
    df = pd.read_csv(ruta_dataset)
    for index, row in df.iterrows():
        anime_name_clean = row['anime']
        capitulo = row['capitulo']
        paginas = ast.literal_eval(row['paginas'])
        
        ruta_capitulo = os.path.join("mangas", anime_name_clean, f'capitulo_{capitulo}')
        os.makedirs(ruta_capitulo, exist_ok=True)
        
        numero_imagen = 1
        lista_imagenes = []
        for url in paginas:
            image_path = descargar_imagen(ruta_capitulo, url, numero_imagen)
            if image_path:
                lista_imagenes.append(image_path)
            numero_imagen += 1
        
        if lista_imagenes:
            crear_pdf(ruta_capitulo, lista_imagenes, anime_name_clean, capitulo)

In [6]:
def main():
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    url = 'https://inmanga.com/ver/manga/Choujin-X/1/00bd17a5-d7fb-48c6-8b39-d6506db9c8cd'

    anime_name = extract_anime_name(url)

    driver.get(url)

    try:
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="ChapList"]'))
        )

        options = select_element.find_elements(By.TAG_NAME, 'option')

        cod = []
        capitulo = []

        for option in options:
            cod.append(option.get_attribute('value'))
            capitulo.append(option.text)

        capitulos_complt = {"capitulo": [], "paginas": []}

        for x, i in zip(capitulo, cod):
            driver.get(extract_segmento_url(url, x, i))

            try:
                select_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="PageList"]'))
                )

                options = select_element.find_elements(By.TAG_NAME, 'option')

                pagina = [opt.text for opt in options]

                boton_next = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, '/html/body/div/section/div/div/div[2]/div[1]/div/div/div[5]/div/button[2]'))
                )

                for _ in range(len(pagina) - 1):
                    boton_next.click()

                url_paginas = []

                for idx in range(1, len(pagina) + 1):
                    try:
                        src_pag = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, f'/html/body/div/section/div/div/div[2]/div[2]/div[1]/a[{idx}]'))
                        )

                        img_element = src_pag.find_element(By.TAG_NAME, 'img')
                        src = img_element.get_attribute('src')

                        url_paginas.append(src)
                    except Exception as e:
                        print(f"Error al procesar el elemento {idx}: {e}")

                capitulos_complt["capitulo"].append(x)
                capitulos_complt["paginas"].append(url_paginas)

            except Exception as e:
                print(f"Error al procesar la página {x}/{i}: {e}")

    finally:
        driver.quit()

    df = pd.DataFrame(capitulos_complt)
    df.insert(0, 'anime', anime_name)

    base_path = os.path.join(os.getcwd(), "mangas", anime_name, "dataset")
    os.makedirs(base_path, exist_ok=True)

    csv_path = os.path.join(base_path, "capitulos_completos.csv")
    df.to_csv(csv_path, index=False)
    print(f"Archivo CSV guardado en: {csv_path}")

    procesar_dataset(csv_path)

    # Mover los PDFs generados y combinarlos
    ruta_pdfs_combinados = mover_pdfs_a_carpeta(base_path)
    combinar_pdfs(ruta_pdfs_combinados,anime_name)

if __name__ == "__main__":
    main()

Archivo CSV guardado en: d:\todo relacionado a mangas\script_descarga\mangas\Choujin X\dataset\capitulos_completos.csv
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\1.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\2.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\3.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\4.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\5.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\6.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\7.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\10.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\11.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\12.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\13.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\14.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\15.jpg
Imagen convertida a JPEG: mangas\Choujin X\capitulo_1\17.jpg
Imagen convertida a JPEG: mangas\C

In [ ]:
df = pd.DataFrame(capitulos_complt)
df.insert(0, 'anime', anime_name)

In [ ]:
base_path = os.path.join(os.getcwd(), "mangas", anime_name, "dataset")
os.makedirs(base_path, exist_ok=True)

In [ ]:
csv_path = os.path.join(base_path, "capitulos_completos.csv")
df.to_csv(csv_path, index=False)
print(f"Archivo CSV guardado en: {csv_path}")

In [ ]:
procesar_dataset(csv_path)

# Mover los PDFs generados y combinarlos
ruta_pdfs_combinados = mover_pdfs_a_carpeta(base_path)
combinar_pdfs(ruta_pdfs_combinados,anime_name)